# CSC_52081_EP Project

Advanced Machine Learning and Autonomous Agents Project

## Introduction

Reinforcement Learning (RL) has emerged as a robust framework for training autonomous agents to learn optimal behaviors through environmental interactions. This study utilizes the [`CarRacing-v3`](https://gymnasium.farama.org/environments/box2d/car_racing/) environment from Gymnasium, which presents a challenging control task in a racing scenario.

### Environment

The environment features a high-dimensional observation space, represented by a $96 \times 96$ RGB image capturing the car and track, necessitating the use of deep convolutional neural networks (CNNs) for effective feature extraction.

#### Action Space

The action space in CarRacing-v3 supports both continuous and discrete control modes.

In **continuous mode**, the agent outputs three real-valued commands:

- steering (ranging from $-1$ to $+1$)
- gas
- braking

In **discrete mode**, the action space is simplified to five actions:

- do nothing
- steer left
- steer right
- gas
- brake

This dual action representation enables a comprehensive evaluation of various RL algorithms under different control settings.

#### Reward

The reward structure combines a penalty of $-0.1$ per frame and a reward of $+\frac{1000}{N}$ for each new track tile visited, where $N$ is the total number of tiles. This incentivizes the agent to balance exploration (visiting tiles) with efficiency (minimizing frame usage). For example, completing the race after visiting all $N$ tiles in 732 frames yields a reward of $1000 - 0.1 \times 732 = 926.8$ points.

### Objective

The primary objective of this project is to compare RL policies across discrete and continuous action modalities. For discrete control, methods like **Deep Q-Network** (DQN) and **SARSA** are implemented, while continuous control is explored using approaches such as the **Cross-Entropy Method** (CEM), **Self-Adaptive Evolution Strategy** (SA-ES), and policy gradient techniques like **Proximal Policy Optimization** (PPO) and **Soft Actor-Critic** (SAC). This comparative analysis aims to understand the strengths and limitations of each method in handling complex decision spaces.

The high-dimensional visual inputs in `CarRacing-v3` require effective feature extraction, addressed through a tailored CNN architecture. Transitioning between discrete and continuous action representations also demands careful algorithmic design and parameter tuning to ensure stable learning and convergence. While prior studies have often focused on either discrete or continuous action spaces separately, this work adopts a comparative approach, evaluating different agents within the same environment to assess performance under similar conditions.

At this stage, the work outlines the methodology and anticipated challenges, focusing on designing the CNN-based feature extractor, implementing RL algorithms, and establishing a framework for performance comparison. Preliminary findings are yet to be finalized, but the study is expected to provide insights into applying RL in high-dimensional, real-time control tasks. Limitations include the preliminary nature of experiments and the need for further tuning and validation. Future work will involve extensive empirical evaluations, exploring additional policy gradient methods, and refining the network architecture to better handle the complexities of `CarRacing-v3`.

### GitHub

The project's code is available on [GitHub](https://github.com/tr0fin0/ensta_CSC_52081_EP_project), offering a reproducible framework for future investigations and extensions.

## Installation

### Environment

#### WSL, Linux or MacOS

A `Python Virtual Environment` will be used for this project by run the following on a terminal on the project folder:

```bash
sudo apt install python3.10-venv
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

### Imports

In [1]:
import datetime
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import torch


from IPython.display import Video
from pathlib import Path
from typing import List

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.ion()

In [3]:
sns.set_context("talk")

In [4]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

### Setup

#### Directories

In [5]:
DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")
DIRECTORY_LOGS = Path(f"{DIRECTORY_OUTPUT}/logs/")

if not DIRECTORY_FIGURES.exists():
    DIRECTORY_FIGURES.mkdir(parents=True)

if not DIRECTORY_MODELS.exists():
    DIRECTORY_MODELS.mkdir(parents=True)

if not DIRECTORY_LOGS.exists():
    DIRECTORY_LOGS.mkdir(parents=True)

print(DIRECTORY_OUTPUT)
print(DIRECTORY_MODELS)
print(DIRECTORY_FIGURES)
print(DIRECTORY_LOGS)

output
output/models
output/images
output/logs


## Demonstration

In [6]:
VIDEO_DEMO = "CSC_52081_EP_demonstration"
(DIRECTORY_FIGURES / f"{VIDEO_DEMO}.mp4").unlink(missing_ok=True)


env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DEMO)


done = False
observation, info = env.reset()

while not done:
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DEMO}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

/home/trofino/git_repositories/classes_ensta/3A/CSC_52081_EP/CSC_52081_EP_project/src/env/lib/python3.10/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/trofino/git_repositories/classes_ensta/3A/CSC_52081_EP/CSC_52081_EP_project/src/dev/output/images folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Description

### Global Definitions

#### Environment

In [14]:
from SkipFrame import SkipFrame


def get_environment_discrete():
    env_discrete = gym.make(
        "CarRacing-v3",
        render_mode="rgb_array",
        continuous=False
    )
    env_discrete = SkipFrame(env_discrete, skip=4)
    env_discrete = gym_wrap.GrayscaleObservation(env_discrete)
    env_discrete = gym_wrap.ResizeObservation(env_discrete, shape=(84, 84))
    env_discrete = gym_wrap.FrameStackObservation(env_discrete, stack_size=4)

    return env_discrete

#### CNN Feature Extractor

### DQN

#### Agent

#### Training

In [ ]:
from Agent_DQN import Agent_DQN
from CNN import CNN
from plot_reward import plot_reward


MODEL_LOAD = False
MODEL_ID = 580000

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCHES = 32
EPISODES = 3000
EPISODES_EXTRA = 300

env_discrete = get_environment_discrete()
state, info = env_discrete.reset()

agent = Agent_DQN(
    state.shape,
    env_discrete.action_space.n,
    DEVICE,
    DIRECTORY_MODELS,
    DIRECTORY_LOGS,
    CNN,
    load_state='train' if MODEL_LOAD else None,
    load_model= f'DQN_{MODEL_ID}.pt' if MODEL_LOAD else None,
    use_DDQN=False
)


episode = EPISODES if MODEL_LOAD else 0
actions = MODEL_ID if MODEL_LOAD else 0
episode_epsilons = []
episode_rewards = []
episode_lengths = []
episode_losses = []
episode_dates = []
episode_times = []

interval_learn = 4      # [actions]
interval_sync = 5000    # [actions]
interval_save = 10000   # [actions]
interval_report = 5000  # [actions]
interval_eval = 50000   # [actions]
interval_log = 10       # [episodes]


while episode <= EPISODES + (EPISODES_EXTRA if MODEL_LOAD else 0):
    episode += 1
    episode_reward = 0
    episode_length = 0

    updating = True
    losses = []
    episode_epsilons.append(agent.epsilon)

    while updating:
        actions += 1
        episode_length += 1

        action = agent.get_action(state)
        new_state, reward, terminated, truncated, info = env_discrete.step(action)
 
        episode_reward += reward
        agent.store(state, action, reward, new_state, terminated)

        state = new_state
        updating = not (terminated or truncated)

        if actions % interval_sync == 0:
            updating_network_parameters = agent.updating_network.state_dict()
            agent.frozen_network.load_state_dict(updating_network_parameters)

        if actions % interval_save == 0:
            agent.save()

        if actions % interval_learn == 0:
            q, loss = agent.update_network(BATCHES)
            losses.append(loss)


    state, info = env_discrete.reset()

    episode_rewards.append(episode_reward)
    episode_lengths.append(episode_length)
    episode_losses.append(np.mean(losses))
    now_time = datetime.datetime.now()
    episode_dates.append(now_time.date().strftime('%Y-%m-%d'))
    episode_times.append(now_time.time().strftime('%H:%M:%S'))

    plot_reward(episode, episode_rewards, actions)

    if episode % interval_log == 0:
        agent.write_log(
            episode_dates,
            episode_times,
            episode_rewards,
            episode_lengths,
            episode_losses,
            episode_epsilons,
        )

agent.save()
agent.write_log(
    episode_dates,
    episode_times,
    episode_rewards,
    episode_lengths,
    episode_losses,
    episode_epsilons,
)
env_discrete.close()

plt.ioff()
plt.show()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

#### Evaluation

In [15]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = 580000


env_discrete = get_environment_discrete()
state, info = env_discrete.reset()

agent = Agent_DQN(
    state.shape,
    env_discrete.action_space.n,
    DEVICE,
    DIRECTORY_MODELS,
    DIRECTORY_LOGS,
    CNN,
    load_state='eval',
    load_model= f'DQN_{MODEL_ID}.pt',
    use_DDQN=False
)
agent.epsilon = 0

scores = []
actions = []
seeds = [i for i in range(2)]

for episode, seed_id in enumerate(seeds):
    state, info = env.reset(seed=seed_id)
    score = 0
    action = 0

    updating = True
    while updating:
        action = agent.get_action(state)
        state, reward, terminated, truncated, info = env.step(action)

        updating = not (terminated or truncated)
        score += reward
        action += 1

    scores.append(score)
    actions.append(action)

    print(f"Episode:{episode}, Score:{score:.2f}, actions: {action}")

env.close()

RuntimeError: Given groups=1, weight of size [16, 4, 8, 8], expected input[1, 96, 96, 3] to have 4 channels, but got 96 channels instead